In [ ]:

import numpy as np 
import pandas as pd 
import seaborn as sns

import fastai
from fastai.vision.all import *
from fastai.metrics import error_rate


**Question 1, 2 & 3**

Chargement du jeu

In [ ]:
path = '/kaggle/input/state-farm-distracted-driver-detection/'
img_list = pd.read_csv(path + 'driver_imgs_list.csv')
img_list.head(1000)

In [ ]:
print("Taille du jeu: ", img_list.shape)


In [ ]:
print("Nombre de conducteurs uniques: ", len(img_list.subject.sort_values().unique())) 


In [ ]:
print("Nombre de classes: ", len(img_list.classname.sort_values().unique())) 


In [ ]:
print("Nombre d'occurences des individus: ", img_list['subject'].value_counts()) 



In [ ]:
occur= img_list.groupby(["subject", "classname"]).size().reset_index(name="occurence")
print(occur.shape[0],"signifie que chaque individu a des images dans les 10 classes") 



# **Ensemble de validation**

### Sélection de 25% pour l'ensemble de validation. 
### Vu que 25% de 26 correspond à 6.5, nous choisirons 7 conducteurs parmi les 26 pour notre ensemble.
### Ici nous pouvons prendre n'importe quel conducteur pour notre division vu qu'ils ont tous des images dans les 10 classes


In [ ]:
valid_subjects = img_list.subject.sort_values().unique()[-7:]

### Creation d'une column is_valid qui affiche True pour les lignes qui seront dans l'ensemble de validation 


In [ ]:

img_list['is_valid'] = img_list['subject'].isin(valid_subjects)

print("Conducteur de l'ensemble de validation: ", valid_subjects)
print(img_list[img_list['is_valid']==True].subject.count())

In [ ]:
img_list

### nouvelle colnne avec class + nom du fichier image


In [ ]:
img_list['img_path'] = img_list.classname + '/' + img_list.img



### Valid_names contient uniquement les noms des images de l'echantillon test


In [ ]:
valid_names = img_list[img_list['subject'].isin(valid_subjects)].img
valid_names = valid_names.to_list()

In [ ]:
img_list

### Chargement des images avec la methode from_df

In [ ]:
dls = ImageDataLoaders.from_df(df=img_list, path = path + 'imgs/train/', fn_col=4, label_col=1, valid_col='is_valid',seed=2021)
dls

**Visualisation des images**

In [ ]:
dls.show_batch()

# Question 4

### Définissons le modèle d'apprentissage(resnet18) et choix du meilleur taux d'apprentissage

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate, model_dir="/tmp/model/")
lrate = learn.lr_find(suggest_funcs=(minimum, steep, valley, slide)) 
lrate 

**Entraînement sur 5 epochs**

In [ ]:
learn.fine_tune(5,base_lr=lrate.minimum, cbs=[ShowGraphCallback(),])

**Question 5**

**Entraînement sur 10 epochs pour rechercher la dégradation**

In [ ]:
learn.fine_tune(10,base_lr=lrate.minimum, cbs=[ShowGraphCallback(),])

La meilleure généralisation se fait au 4ème epoch, après cela il y a une dégradation 

**Question 6**

In [ ]:
transform = [Rotate(360, p=1, mode='bilinear'), Brightness(max_lighting=0.9, p=0.35),Contrast(max_lighting=0.4, p=0.25),
              RandomErasing(p=0.2, sl=0.0, sh=0.2, min_aspect=0.3, max_count=2), Flip(p=1), 
             Zoom(max_zoom=1.1,p=0.1), RandomResizedCrop(224)]

dls2 = ImageDataLoaders.from_df(df=img_list, path = path + 'imgs/train/', fn_col=4, label_col=1, valid_col='is_valid',seed=2021, batch_tfms=transform)


dls2.show_batch()

In [ ]:
learn2 = cnn_learner(dls2, resnet18, metrics=error_rate, model_dir="/tmp/model/")
lrate2 = learn2.lr_find(suggest_funcs=(minimum, steep, valley, slide))
lrate2

In [ ]:
learn2.fine_tune(epochs=5, base_lr=lrate2.minimum, cbs=[ShowGraphCallback(),])

**Question 7**

Un problème de mémoire survient lorsque nous exécutons les codes ci-dessous

In [ ]:
result = ClassificationInterpretation.from_learner(learn2)

In [ ]:
result.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
result.print_classification_report()

Question 8

In [ ]:
result.most_confused(min_val=50)